In [1]:
import os
import sys
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import keras.backend as K
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
from keras.optimizers import SGD, Adam
from keras.models import Sequential
from keras.regularizers import l1_l2
from sklearn.metrics import accuracy_score
from collections import OrderedDict, Counter
from tensorflow.python.ops import gen_array_ops
from keras.layers import Dense, Dropout, Flatten
from names import Names as n

In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)
pd.set_option('display.max_colwidth', None)
matplotlib.use("tkagg")
%matplotlib inline

In [3]:
# df1 = pd.read_csv("models/cfu_neural_network/df_for_testing.csv")
# df2 = pd.read_csv("models/cfu_neural_network/df_for_testing_2.csv")
# df3 = pd.read_csv("experiment_data/processed/yeast_ethanol/pipeline_data.csv")
# df4 = pd.read_csv("experiment_data/processed/yeast_ethanol/sampled_pipeline_data.csv")
df5 = pd.read_csv("experiment_data/raw/sytoxYeastSTATES-LiveDeadClassification-placeholder/pipeline_data.csv")
# df6 = pd.read_csv(
#     "experiment_data/raw/Duke-YeastSTATES-Ethanol-Time-Series-LiveDeadClassification/df_for_testing_duke.csv")
# df7 = pd.read_csv(
#     "experiment_data/raw/Duke-YeastSTATES-Ethanol-Time-Series-LiveDeadClassification/processed_duke_data.csv")

df1: Combined Data - Sampled (259 thousand events) - Columns consolidated - Features are only morph columns

df2: Duke Data - All (2.86 million events) - All Features

df3: Combined Data - All (10.7 million events) - Columns NOT consolidated - All Features

df4: Combined Data - Sampled (536 thousand events) - Columns NOT consolidated - All Features

df5: Strateos Data - All (7.87 million events) - All Features

df6: Duke Data - All (2.86 million events) - All Features

df7: Duke Data - All (2.86 million events) - All Features - Now includes stain column

In [ ]:
display(df1)
print()
display(df1[n.inducer_concentration].value_counts().sort_index())

In [ ]:
display(df2)
print()
display(df2[n.inducer_concentration].value_counts().sort_index())

In [ ]:
display(df3)
print()
display(df3[n.inducer_concentration].value_counts().sort_index())

In [ ]:
display(df4)
print()
display(df4[n.inducer_concentration].value_counts().sort_index())

In [4]:
display(df5)
print()
display(df5[n.ethanol].value_counts().sort_index())

,arbitrary_index,stain,ethanol,time_point,FSC-A,SSC-A,BL1-A,RL1-A,FSC-H,SSC-H,BL1-H,RL1-H,FSC-W,SSC-W,BL1-W,RL1-W,log_FSC-A,log_FSC-H,log_FSC-W,log_SSC-A,log_SSC-H,log_SSC-W,log_RL1-A,log_RL1-H,log_RL1-W,log_BL1-A,log_BL1-H,log_BL1-W
0,0,1,0.0,12,1048575.0,1048575.0,61.0,2196.0,92697.0,81962.0,106.0,156.0,1023.0,1023.0,0.0,0.0,6.020599,4.967066,3.009876,6.020599,4.913613,3.009876,3.341632,2.193125,0.0,1.785330,2.025306,0.0
1,1,1,0.0,12,75859.0,35377.0,-12.0,134.0,61710.0,28229.0,78.0,98.0,39.0,30.0,0.0,0.0,4.880007,4.790356,1.591065,4.548721,4.450695,1.477121,2.127105,1.991226,0.0,0.000000,1.892095,0.0
2,2,1,0.0,12,85090.0,42649.0,23.0,4.0,66049.0,33003.0,42.0,115.0,41.0,33.0,0.0,0.0,4.929879,4.819866,1.612784,4.629909,4.518553,1.518514,0.602060,2.060698,0.0,1.361728,1.623249,0.0
3,3,1,0.0,12,86520.0,48336.0,50.0,87.0,70961.0,38604.0,57.0,103.0,40.0,34.0,0.0,0.0,4.937117,4.851020,1.602060,4.684271,4.586632,1.531479,1.939519,2.012837,0.0,1.698970,1.755875,0.0
4,4,1,0.0,12,47880.0,36321.0,-1.0,115.0,46767.0,35206.0,50.0,108.0,30.0,27.0,0.0,0.0,4.680154,4.669940,1.477121,4.560158,4.546617,1.431364,2.060698,2.033424,0.0,0.000000,1.698970,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7875721,7875721,1,10.0,11,56074.0,22842.0,50.0,-63.0,49427.0,19958.0,75.0,133.0,33.0,22.0,0.0,0.0,4.748762,4.693964,1.518514,4.358734,4.300117,1.342423,0.000000,2.123852,0.0,1.698970,1.875061,0.0
7875722,7875722,1,10.0,11,56830.0,19448.0,-72.0,-91.0,56640.0,18720.0,58.0,196.0,32.0,18.0,0.0,0.0,4.754578,4.753123,1.505150,4.288875,4.272306,1.255273,0.000000,2.292256,0.0,0.000000,1.763428,0.0
7875723,7875723,1,10.0,11,51855.0,21187.0,64.0,-77.0,41082.0,17528.0,94.0,225.0,36.0,21.0,0.0,0.0,4.714791,4.613652,1.556303,4.326069,4.243732,1.322219,0.000000,2.352183,0.0,1.806180,1.973128,0.0
7875724,7875724,1,10.0,11,46824.0,15689.0,52.0,-37.0,43634.0,15019.0,91.0,153.0,31.0,16.0,0.0,0.0,4.670469,4.639825,1.491362,4.195595,4.176641,1.204120,0.000000,2.184691,0.0,1.716003,1.959041,0.0


0.0     2191947
10.0    1556805
15.0    1125385
20.0    1548106
80.0    1453483
Name: ethanol, dtype: int64

In [ ]:
display(df6)
print()
display(df6[n.inducer_concentration].value_counts().sort_index())

In [ ]:
display(df7)
print()
display(df7[n.inducer_concentration].value_counts().sort_index())

# CFU Means from Duke 2.0 Data

In [5]:
cfu_means = pd.read_csv("cfu_means_v2.csv")

## Strateos Data

In [16]:
strateos = df5.copy()
strateos.rename(columns = {"ethanol": n.inducer_concentration, "time_point": n.timepoint}, inplace=True)
strateos[n.timepoint] = strateos[n.timepoint]/2

In [17]:
strateos[n.inducer_concentration].value_counts()

0.0     2191947
10.0    1556805
20.0    1548106
80.0    1453483
15.0    1125385
Name: inducer_concentration, dtype: int64

In [18]:
strateos[n.timepoint].value_counts()

5.5    865777
5.0    832877
0.5    826251
6.0    826251
4.5    790411
4.0    641340
1.5    583861
3.5    545648
1.0    545325
3.0    488693
2.0    482829
2.5    446463
Name: timepoint, dtype: int64

In [19]:
# might want to change to left join
strateos = pd.merge(strateos, cfu_means, how="inner", on=["inducer_concentration", "timepoint"])
strateos.sort_values(by=["inducer_concentration", "timepoint"], inplace=True)

In [20]:
strateos[n.timepoint].value_counts()

0.5    826251
6.0    826251
3.0    488693
Name: timepoint, dtype: int64

In [10]:
strateos.drop(columns="arbitrary_index", inplace=True)

In [11]:
strateos.drop(columns = ["FSC-A", "SSC-A", "BL1-A", "RL1-A", 
                         "FSC-H", "SSC-H", "BL1-H", "RL1-H", 
                         "FSC-W", "SSC-W", "BL1-W", "RL1-W"], inplace=True)

In [12]:
strateos.columns = strateos.columns.str.lstrip('log_')

In [13]:
strateos["source"] = "strateos"

In [14]:
strateos

,stain,inducer_concentration,timepoint,FSC-A,FSC-H,FSC-W,SSC-A,SSC-H,SSC-W,RL1-A,RL1-H,RL1-W,BL1-A,BL1-H,BL1-W,percent_live,source
826251,1,0.0,0.5,6.020599,4.967066,3.009876,6.020599,4.913613,3.009876,3.341632,2.193125,0.0,1.785330,2.025306,0.0,98.776667,strateos
826252,1,0.0,0.5,4.880007,4.790356,1.591065,4.548721,4.450695,1.477121,2.127105,1.991226,0.0,0.000000,1.892095,0.0,98.776667,strateos
826253,1,0.0,0.5,4.929879,4.819866,1.612784,4.629909,4.518553,1.518514,0.602060,2.060698,0.0,1.361728,1.623249,0.0,98.776667,strateos
826254,1,0.0,0.5,4.937117,4.851020,1.602060,4.684271,4.586632,1.531479,1.939519,2.012837,0.0,1.698970,1.755875,0.0,98.776667,strateos
826255,1,0.0,0.5,4.680154,4.669940,1.477121,4.560158,4.546617,1.431364,2.060698,2.033424,0.0,0.000000,1.698970,0.0,98.776667,strateos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505646,1,80.0,6.0,4.121822,3.756712,0.000000,2.887617,2.685742,0.000000,0.000000,1.732394,0.0,1.431364,1.806180,0.0,0.000000,strateos
505647,1,80.0,6.0,4.981198,3.822037,0.000000,0.000000,2.838849,0.000000,0.000000,2.025306,0.0,0.000000,2.071882,0.0,0.000000,strateos
505648,1,80.0,6.0,5.295772,3.789792,0.000000,0.000000,2.615950,0.000000,0.000000,2.086360,0.0,2.437751,1.963788,0.0,0.000000,strateos
505649,1,80.0,6.0,5.471633,3.768120,0.000000,0.000000,2.840106,0.000000,0.000000,2.127105,0.0,1.748188,1.982271,0.0,0.000000,strateos


## Duke Data

In [ ]:
duke = df7.copy()
duke = duke[['stain'] + [col for col in duke.columns if col != 'stain']]
duke["stain"] = duke["stain"].astype(int)
duke["source"] = "duke"

In [ ]:
duke

## Sample Both Duke and Strateos Data
Actually only Strateos since we saw Duke data had gain issues, and now we have Duke 2.0 data instead. So I deleted cells with Duke data in this notebook.

In [41]:
num_samples = 27000

### Sample Strateos

In [42]:
strateos_sample = strateos.groupby(["stain", n.inducer_concentration, n.timepoint]).sample(n=num_samples,
                                                                                           random_state=5)

In [43]:
strateos_sample

,stain,inducer_concentration,timepoint,FSC-A,FSC-H,FSC-W,SSC-A,SSC-H,SSC-W,RL1-A,RL1-H,RL1-W,BL1-A,BL1-H,BL1-W,percent_live,source
907261,0,0.0,0.5,4.962128,4.835532,1.623249,4.612466,4.498986,1.505150,0.000000,1.505150,0.0,0.000000,1.556303,0.0,98.776667,strateos
957565,0,0.0,0.5,4.347993,4.460958,1.361728,4.213093,4.226651,1.230449,0.000000,1.556303,0.0,0.000000,1.491362,0.0,98.776667,strateos
999780,0,0.0,0.5,5.035678,4.918240,1.643453,4.677013,4.573823,1.518514,0.000000,2.017033,0.0,1.113943,1.986772,0.0,98.776667,strateos
959500,0,0.0,0.5,4.912971,4.819478,1.612784,4.612561,4.500922,1.518514,2.161368,2.041393,0.0,2.176091,1.929419,0.0,98.776667,strateos
951703,0,0.0,0.5,4.625929,4.624292,1.462398,4.359741,4.360688,1.342423,1.414973,1.770852,0.0,1.113943,1.602060,0.0,98.776667,strateos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491475,1,80.0,6.0,4.256164,3.546172,0.000000,0.000000,2.257679,0.000000,0.000000,1.763428,0.0,1.949390,1.755875,0.0,0.000000,strateos
435566,1,80.0,6.0,3.509337,3.339451,0.000000,0.301030,2.887617,0.000000,0.000000,2.012837,0.0,1.414973,1.908485,0.0,0.000000,strateos
484534,1,80.0,6.0,4.814527,3.656098,0.000000,0.000000,2.747412,0.000000,2.654177,2.158362,0.0,2.357935,2.012837,0.0,0.000000,strateos
442691,1,80.0,6.0,3.364176,3.334051,0.000000,3.061075,3.186674,0.000000,2.133539,2.416641,0.0,0.000000,1.770852,0.0,0.000000,strateos


In [44]:
strateos_sample.groupby([n.inducer_concentration, n.timepoint]).size()

inducer_concentration  timepoint
0.0                    0.5          54000
                       3.0          54000
                       6.0          54000
10.0                   0.5          54000
                       3.0          54000
                       6.0          54000
15.0                   0.5          54000
                       3.0          54000
                       6.0          54000
20.0                   0.5          54000
                       3.0          54000
                       6.0          54000
80.0                   0.5          54000
                       3.0          54000
                       6.0          54000
dtype: int64

## Write out DataFrames

In [45]:
# strateos_sample.to_csv("experiment_data/processed/final_strateos_sample.csv", index=False)